
<h1 style="color:blue; font-size:24px;">Importing Required Libraries</h1>

This section imports libraries for GUI creation (tkinter), data handling (pandas, numpy), machine learning (scikit-learn), visualization (matplotlib, folium), and system operations (os, webbrowser).

In [26]:
import tkinter as tk
from tkinter import messagebox
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from geopy.geocoders import Nominatim
import folium
import webbrowser
import os

<h2 style="color:black; font-size:24px;">Setting API Details</h2>
This section defines the API key and base URL for interacting with the OpenWeatherMap API. 

In [27]:
API_KEY = '2ff8cba845e3bf73c7780eea1b6b5402'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'

<h2 style="color:black; font-size:24px;">Fetch Current Weather Data</h2>
This function retrieves the current weather details for a specified city using the OpenWeather API, including temperature, humidity, wind speed, pressure, and more, and returns the data in a structured format.

In [28]:
# Fetch current weather
def get_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    return {
        'city': data['name'],
        'current_temp': round(data['main']['temp']),
        'feels_like': round(data['main']['feels_like']),
        'temp_min': round(data['main']['temp_min']),
        'temp_max': round(data['main']['temp_max']),
        'humidity': round(data['main']['humidity']),
        'description': data['weather'][0]['description'],
        'country': data['sys']['country'],
        'Wind_Gust_Speed': round(data['wind']['speed']),
        'pressure': round(data['main']['pressure']),
        'lat': data['coord']['lat'],
        'lon': data['coord']['lon']
    }


<h2 style="color:black; font-size:24px;">Prepare Data and Train Random Forest Regressor for Prediction</h2>
Preparing regression data by creating feature and target sets and then train a Random Forest Regressor model using the prepared data to make predictions based on the feature.

In [29]:
# Train RandomForest Regressor for prediction
def prepare_regression_data(data, feature):
    X, y = [], []
    for i in range(len(data) - 1):
        X.append(data[feature].iloc[i])
        y.append(data[feature].iloc[i + 1])
    X = np.array(X).reshape(-1, 1)
    y = np.array(y)
    return X, y

def train_regression_model(X, y):
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X, y)
    return model

<h2 style="color:black; font-size:24px;">Predict Future Values</h2>
The predict_future function generates predictions for the next 5 time intervals using a trained model, while the moving_average function smooths the predictions by applying a moving average technique with a specified window size.

In [30]:
# Predict future values using the regression model
def predict_future(model, current_value):
    predictions = [current_value]
    for i in range(1, 6):  # Predicting next 5 time intervals
        next_value = model.predict(np.array([[predictions[-1]]]))
        predictions.append(next_value[0])
    return predictions[1:]

# Moving Average function to smooth predictions
def moving_average(data, window_size=3):
    return np.convolve(data, np.ones(window_size) / window_size, mode='valid')

<h2 style="color:black; font-size:24px;"> Create and Display a City Map</h2>
Generates an interactive map centered on the given city coordinates using Folium, adds a marker for the city, saves the map as an HTML file, and opens it in the default browser.

In [31]:
# Function to create map of city
def create_map(city, lat, lon):
    m = folium.Map(location=[lat, lon], zoom_start=10)
    folium.Marker([lat, lon], popup=city).add_to(m)

    # Save the map to an HTML file
    map_path = "city_map.html"
    m.save(map_path)

    # Open map in the default browser
    webbrowser.open(f'file://{os.path.realpath(map_path)}')

    return map_path


<h2 style="color:black; font-size:24px;"> Display Current Weather Information</h2>
This function retrieves and displays the current weather data for a city entered by the user, including details like temperature, humidity, wind speed, and more, handling errors if the weather data cannot be fetched.

In [32]:
# Function to display current weather
def show_current_weather():
    city = city_entry.get()
    try:
        # Get current weather
        weather = get_current_weather(city)

        # Display weather info
        result_text = f"City: {weather['city']}, {weather['country']}\n"
        result_text += f"CW Temperature: {weather['current_temp']}°C\n"
        result_text += f"Feels Like: {weather['feels_like']}°C\n"
        result_text += f"Min Temperature: {weather['temp_min']}°C\n"
        result_text += f"Max Temperature: {weather['temp_max']}°C\n"
        result_text += f"Humidity: {weather['humidity']}%\n"
        result_text += f"Description: {weather['description']}\n"
        result_text += f"Wind Gust Speed: {weather['Wind_Gust_Speed']} m/s\n"
        result_text += f"Pressure: {weather['pressure']} hPa\n"
        result_label.config(text=result_text)
    except Exception as e:
        messagebox.showerror("Error", str(e))


<h2 style="color:black; font-size:24px;"> Weather Prediction and Visualization Application</h2>
This application allows users to input a city name and fetches current weather data, including temperature and humidity. It uses regression models to predict future values for temperature and humidity, applies a moving average to smooth predictions, and visualizes the data in a graph. Additionally, it displays the city’s location on a map and integrates this functionality within a Tkinter-based GUI.

In [ ]:
# Function to display prediction
def show_prediction():
    city = city_entry.get()
    try:
        # Get current weather
        weather = get_current_weather(city)

        # Load weather data for predictions (using sample CSV or historical data)
        data = pd.read_csv('weather.csv')  # Assuming you have a weather dataset
        X_temp, y_temp = prepare_regression_data(data, 'Temp')  # Temperature data for prediction
        X_hum, y_hum = prepare_regression_data(data, 'Humidity')  # Humidity data for prediction

        # Train regression models for temperature and humidity
        temp_model = train_regression_model(X_temp, y_temp)
        hum_model = train_regression_model(X_hum, y_hum)

        # Predict future values
        future_temp = predict_future(temp_model, weather['current_temp'])
        future_humidity = predict_future(hum_model, weather['humidity'])

        # Smooth predictions using moving average
        smoothed_temp = moving_average(future_temp)
        smoothed_humidity = moving_average(future_humidity)

        # Unified graph for temperature and humidity with smooth predictions
        fig, ax = plt.subplots(figsize=(8, 4))  # Reduced size for more accuracy

        # Combine current and predicted data into a single line graph
        all_temp_data = [weather['current_temp']] + smoothed_temp.tolist()
        all_hum_data = [weather['humidity']] + smoothed_humidity.tolist()

        # Plot the continuous line graph for both Temperature and Humidity
        ax.plot(range(len(all_temp_data)), all_temp_data, label='Temperature (°C)', color='tab:red', linestyle='-', marker='o', markersize=6)
        ax.plot(range(len(all_hum_data)), all_hum_data, label='Humidity (%)', color='tab:blue', linestyle='-', marker='o', markersize=6)

        # Change the color of the current weather dots
        ax.plot(0, weather['current_temp'], 'o', color='green', markersize=8, label="Current Temp")
        ax.plot(0, weather['humidity'], 'o', color='purple', markersize=8, label="Current Humidity")
        # Add text annotations for current values
        ax.text(0, weather['current_temp']+ 2, 'CT', color='green', fontsize=10, verticalalignment='bottom', horizontalalignment='center')
        ax.text(0, weather['humidity']-2, 'CH', color='purple', fontsize=10, verticalalignment='top', horizontalalignment='center')

        ax.set_title("Current and Predicted Temperature and Humidity", fontsize=16, fontweight='bold')
        ax.set_xlabel("Time (Future Prediction)", fontsize=12, fontweight='bold')
        ax.set_ylabel("Temperature (°C) / Humidity (%)", fontsize=12, fontweight='bold')
        ax.legend()

        # Create a canvas and display the plot
        prediction_window = tk.Toplevel(root)
        prediction_window.title("Weather Predictions")

        title_label = tk.Label(prediction_window, text="Weather Prediction", font=("Helvetica", 16, 'bold'), fg="#1e90ff")
        title_label.pack(pady=10)

        canvas = FigureCanvasTkAgg(fig, master=prediction_window)
        canvas.draw()
        canvas.get_tk_widget().pack(side=tk.LEFT, padx=10)

    except Exception as e:
        messagebox.showerror("Error", str(e))

# Function to display location (Map)
def show_location():
    city = city_entry.get()
    try:
        # Get current weather to fetch lat and lon
        weather = get_current_weather(city)

        # Create the map and open in the browser
        create_map(weather['city'], weather['lat'], weather['lon'])

    except Exception as e:
        messagebox.showerror("Error", str(e))

# GUI Setup
root = tk.Tk()
root.title("Weather Prediction App")
root.geometry("500x600")  # Adjusting the window height to take up half of the screen
root.config(bg="#f5f5f5")

# Title Label
title_label = tk.Label(root, text="Weather Prediction Application", font=("Helvetica", 18, 'bold'), fg="#1e90ff", bg="#f5f5f5")
title_label.pack(pady=20)

# Input field
tk.Label(root, text="Enter City Name:", font=("Helvetica", 12), bg="#f5f5f5").pack()
city_entry = tk.Entry(root, font=("Helvetica", 12))
city_entry.pack(pady=10)

# Buttons
button_style = {'font': ("Helvetica", 12, 'bold'), 'width': 25, 'height': 2, 'bg': '#1e90ff', 'fg': 'white'}

tk.Button(root, text="Show Current Weather", command=show_current_weather, **button_style).pack(pady=5)
tk.Button(root, text="Show Prediction", command=show_prediction, **button_style).pack(pady=5)
tk.Button(root, text="Show Location", command=show_location, **button_style).pack(pady=5)

# Result Display
result_label = tk.Label(root, text="", font=("Helvetica", 12), bg="#f5f5f5", justify="left")
result_label.pack(pady=10)

root.mainloop()
